In [1]:
import numpy as np

# Problems 1-6

In [372]:
class LinearOpimization:
    
    def __init__(self, c, A, b):
        self.c = c
        self.A = A
        self.b = b
        if np.any(self.A @ np.zeros(len(self.c)) > self.b):
            raise ValueError("Not feasible at the origin")
        self.m = len(self.b)
        self.n = len(self.c)
        basic = np.arange(self.n, self.n + self.m)
        nonbasic = np.arange(0, self.n)
        self.L = np.concatenate((basic, nonbasic))
        self.basic = self.L[:self.m]
        self.nonbasic = self.L[self.m:]
        self.T = self.initial_tableau()
        
    def initial_tableau(self):
        c_bar = np.hstack((self.c, np.zeros(self.m)))
        A_bar = np.hstack((self.A, np.identity(self.m)))
        T_top = np.hstack((0, -1 * c_bar, 1))
        T_bottom = np.column_stack((self.b, A_bar, np.zeros(self.m)))
        return np.vstack((T_top, T_bottom))
    
    def pivot_ij(self):
        j = np.argwhere(self.T[0, 1:-1] < 0)[0][0] + 1
        col_j = self.T[1:, j].copy()
        if np.all(col_j <= 0):
            raise ValueError("The problem is unbounded.")
        non_pos = col_j <= 0
        col_0 = self.T[1:, 0].copy()
        col_j[non_pos] = np.nan
        i = np.nanargmin(col_0 / col_j) + 1
        return i, j
    
    def pivot(self):
        i, j = self.pivot_ij()
        temp = self.L[i-1]
        self.L[i - 1] = self.L[self.m + j - 1]
        self.L[self.m + j - 1] = temp
        
        self.T[i] /= self.T[i, j]
        row_i = self.T[i]
        for k, row in enumerate(self.T):
            if k == i:
                pass
            else:
                mult = -row[j]
                self.T[k] = row_i * mult + row
                
    def solve(self):
        while np.any(self.T[0] < 0):
            self.pivot()
        basic_dict = {self.basic[i]: round(self.T[i+1, 0], 2) for i in range(self.m)}
        nonbasic_dict = {self.nonbasic[i]: 0 for i in range(self.n)}
        return self.T[0, 0], basic_dict, nonbasic_dict
            

In [373]:
c = np.array([3, 2])
A = np.array([[1, -1], [3, 1], [4, 3]])
b = np.array([2, 5, 7])
opt = LinearOpimization(c, A, b)

In [374]:
opt.solve()

(5.2, {0: 1.6, 1: 0.2, 2: 0.6}, {3: 0, 4: 0})

# Problem 7

In [382]:
data = np.load('productMix.npz')

In [383]:
A = data['A']
p = data['p']
m = data['m']
d = data['d']

In [384]:
A = np.vstack((A, np.identity(len(p))))
b = np.hstack((m, d))

In [385]:
LinearOpimization(p, A, b).solve()

(7453.596491228071,
 {0: 10.0, 1: 6.19, 2: 12.0, 3: 1.79, 6: 0.97, 8: 13.81, 10: 8.21},
 {4: 0, 5: 0, 7: 0, 9: 0})